In [ ]:
import sys
sys.path.append("../")
from broncode.apple_classifyer import AppleClassifyer

import torchvision.transforms as T
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional
from torchvision.datasets import ImageFolder
from torchvision.io import read_image, ImageReadMode
from torchvision import models
import torch
import pickle as pkl
import os
from PIL import Image


<h1>Time to test a predefined model and compare it to my own, for this I chose Resnet50</h1>

In [ ]:
# Paths
dataset_path = "../informatie/apple_disease_classification/images/Train/Dataset/"
apple_model_path = "../models/AppleClassifyer_86"

In [ ]:
# Dataset class
class DatasetAppels(Dataset):
    def __init__(self, img_folder_path, transform):

        image_folder = ImageFolder(img_folder_path, transform=transform)
        self.images = [image[0] for image in image_folder]
        self.labels = image_folder.targets
        self.class_dict = image_folder.class_to_idx

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return [self.images[idx], self.labels[idx]]
        

In [ ]:
# From torchvision.transforms I use Compose() to create a list of transformations, Resnet50 wants images to be 224,224 and a mean as shown below
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create dataset and apply transformations
dataset = DatasetAppels(dataset_path, preprocess)

In [ ]:
# Create dataloaders
generator1 = torch.Generator().manual_seed(13)

# Because the images are larger than before I decide to reduce the size of my train loader by a large amount to speed up the training process and make it lighter.
train_dataset, test_dataset, val_dataset, _ = random_split(dataset, [0.2, 0.2, 0.2, 0.4], generator=generator1)
print(len(train_dataset), len(test_dataset), len(val_dataset))

# Create train, test and val dataloaders for later use.
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

In [ ]:
# Create a class for the Resnet model
class Resnet50():
    def __init__(self): 
        # Import the Resnet50 model with pretrained weights  
        self.model = models.resnet50(weights='IMAGENET1K_V1')
        num_ftrs = self.model.fc.in_features

        # Set number of output classes, normaly resnet has about a 1000 but I only need 4
        self.model.fc = nn.Linear(num_ftrs, 4)
        self.model = self.model.to("cuda")

        # Determine loss function
        self.loss_func = nn.CrossEntropyLoss()

        

    # For the training and other functions I copypasted them from my AppleClassifyer() and with a few tweaks I got it working pretty quick
    # Function for cuda
    def cuda_available(self):

        if torch.cuda.is_available():
            return torch.device("cuda")
        else:
            return torch.device("cpu")
        
    # Create a function to calculate the loss
    def loss_calc(self, batch):
        image = batch[0].to(self.cuda_available())
        labels = batch[1].to(self.cuda_available())
        pred = self.model(image)        
        loss = self.loss_func(pred, labels)
        return loss
    
    # Create function to evaluate the accuracy of my model
    def evaluate_accuracy(self, test_loader):
        cor_pred = 0
        bad_pred = 0
        for batch in test_loader:
            image, labels = batch
            image = image.to(self.cuda_available())
            labels = labels.to(self.cuda_available())
            pred = self.model(image)
    
            _, y_pred = torch.max(pred,1)

            for image, labels in zip(y_pred, labels):
                if image == labels:
                    cor_pred += 1
                else:
                    bad_pred += 1

        acc = cor_pred/(cor_pred + bad_pred) * 100
        return acc

    def fit(self, train_loader, val_loader, test_loader, epochs, lr, opt_function=torch.optim.Adam):
        
        optimizer = opt_function(self.model.parameters(),lr )
        # Set to cuda (gpu)
        self.model.to(self.cuda_available())
        # Create empty list to save the validation results
        history = []
        # Training loop
        for epoch in range(epochs):
            print("epoch:",epoch+1)
            self.model.train()
            
            for batch in train_loader:
                optimizer.zero_grad()
                # I use the function loss_calc that I created below so that I can use this at the validation.
                loss = self.loss_calc(batch)
                loss.backward()
                optimizer.step()

            # Validate
            with torch.no_grad():    
                self.model.eval()
                val_loss = []

                for batch in val_loader:
                    loss = self.loss_calc(batch)
                    val_loss.append(loss)  
            
            # Append the sum(val_loss)
            history.append(sum(val_loss))
            print(sum(val_loss))        
        
        # I use a self made function return a percentage of the accuracy
        acc = round(self.evaluate_accuracy(test_loader))
        print("Accuracy:",acc)
        return history, acc
    
    # Function to predict a single image
    def predict_image(self, image):
        image = image.to(self.cuda_available())
        pred = self.model(image)

        _, y_pred = torch.max(pred,1)                
        result = y_pred

        return result

In [ ]:
# Start training
model = Resnet50()
history, acc = model.fit(train_loader, val_loader, test_loader, lr = 0.00001, epochs=15)

In [ ]:
# Use pickle to save the Resnet50 model
model_path = f"..//models/Resnet50_{acc}"

with open(model_path, 'wb') as f:
        pkl.dump(model, f)

In [ ]:
# Load model using pickle
load = open(apple_model_path, "rb")
apple_classifyer = pkl.load(load)

In [ ]:
# Lets compare them!
resize = T.Resize((64,64))

result_resnet = []
result_selfmade = []

# My test images are numbers, so with this list I can use a for loop to call in every image I want
normal = [9, 10, 11, 21, 22, 23, 32, 33, 34, 44, 45, 46, 56]
print(dataset.class_dict)
print("\n")
for i in normal:
    test_path = f"../informatie/apple_disease_classification/images/Test/Normal_Apple/{i}.jpg"

    test_img = Image.open(test_path)
    test_img = preprocess(test_img)

    test_img2 = read_image(test_path, ImageReadMode.RGB)/255
    test_img2 = resize(test_img2)

    test_img = test_img.unsqueeze(0)
    test_img2 = test_img2.unsqueeze(0)

    result_resnet.append(model.predict_image(test_img).item())
    result_selfmade.append(apple_classifyer.predict_image(test_img2).item())

# Check the amount of good results
good_resnet = 0
for i in range(len(result_resnet)):
    if result_resnet[i] == 1:
        good_resnet += 1

good_selfmade = 0
for i in range(len(result_selfmade)):
    if result_selfmade[i] == 1:
        good_selfmade += 1

# Let's compare!
print(f"Resnet50: {result_resnet}")
print(f"Selfmade: {result_selfmade}")
print("\n")
print(f"Correct by Resnet50: {good_resnet}/{len(result_resnet)}")
print(f"Correct by AppleClassifyer: {good_selfmade}/{len(result_selfmade)}")


<h3>Interestingly my own model (AppleClassifyer) performs better on these 13 normal apples than the Resnet50 model even though the Resnet50 model has a better accuracy overall.</h3>